In [2]:
import os
import math

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm, transforms
from mpl_axes_aligner import align
import numpy as np
np.seterr(divide='ignore')
import pandas as pd
from tqdm import tqdm
from scipy import special
from scipy.stats import norm
from scipy.stats import multivariate_normal
import h5py
import torch
from torchviz import make_dot

import wf_func as wff

In [3]:
def normcombine(x, m, s, a):
    return a[0] * norm.pdf((x - m[0]) / s[0]) + a[1] * norm.pdf((x - m[1]) / s[1])

def normcombine2d(x, m, s, a, rho):
    return a[0, 0] * multivariate_normal.pdf(x, mean=[m[0, 0], m[1, 0]], cov=matrix(s[0, 0], s[1, 0], rho[0, 0])) + a[0, 1] * multivariate_normal.pdf(x, mean=[m[0, 0], m[1, 1]], cov=matrix(s[0, 0], s[1, 1], rho[0, 1])) + a[1, 0] * multivariate_normal.pdf(x, mean=[m[0, 1], m[1, 0]], cov=matrix(s[0, 1], s[1, 0], rho[1, 0])) + a[1, 1] * multivariate_normal.pdf(x, mean=[m[0, 1], m[1, 1]], cov=matrix(s[0, 1], s[1, 1], rho[1, 1]))

def matrix(sx, sy, rho):
    return np.array([[sx ** 2, rho * sx * sy], [rho * sx * sy, sy ** 2]])

def chargehist(t):
    c = norm.pdf(t, loc=160., scale=40.)
#     q1 = 150.8
#     sigma = 37.59
#     w = 2.433e-5
#     alpha = 0.01335
#     mu = 2.851e-5
#     c = np.exp(-mu)*(w*alpha*np.exp(-alpha*t))
#     c = c + mu*np.exp(-mu)*(
#         (1-w)/(sigma*np.sqrt(2*np.pi))*np.exp(-(t-q1)**2/(2*sigma**2))+
#         w*(alpha/2*np.exp(-alpha*(t-q1-alpha/2*sigma**2))*(1+special.erf(t-q1-alpha*sigma**2)/(np.sqrt(2)*sigma))))
    return c

In [4]:
Thres = {'xiaopeip':0.2, 'lucyddm':0.2, 'fftrans':0.1, 'findpeak':0.1, 'threshold':0.1, 'omp':0}
std = 1.
spe_pre = wff.read_model('spe.h5', 1)
window = 1029
p = spe_pre[0]['parameters']
t = np.arange(window).astype(np.float)
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
t = np.arange(0, 100, 0.1)
ax.plot(t, wff.spe(t, p[0], p[1], p[2]), color='b', label='Single PE response')
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.grid()
ax.set_xlim(0, 80)
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax.legend()
fig.savefig('Note/figures/spe.pgf')
fig.savefig('Note/figures/spe.pdf')
plt.close()

In [20]:
fig = plt.figure(figsize=(8, 6))
t = np.arange(-4 * 5, 5 * 20, 0.1)
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.95, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(t, wff.convolve_exp_norm(t, 20, 0), label=r'$(20,0)$', color='g')
ax.plot(t, wff.convolve_exp_norm(t, 0, 5), label=r'$(0,5)$', color='r')
ax.plot(t, wff.convolve_exp_norm(t, 20, 5), label=r'$(20,5)$', color='b')
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.grid()
ax.set_xlim(xmin=-4 * int(5))
ax.set_ylabel(r'$\mathrm{PDF}$')
ax.legend(title=r'$(\tau, \sigma)/\si{ns}$', loc='upper right')
ax.set_ylim(0, ax.get_ylim()[1] * 1.05)
ax.annotate(r'$t_{0}$', xy=(0, 0), xytext=(5, 0.01), arrowprops=dict(facecolor='k', shrink=0.1, width=0.1, headwidth=2))
fig.savefig('Note/figures/profile.pgf')
fig.savefig('Note/figures/profile.pdf')
plt.close()

In [10]:
Mu = 5
Tau = 20
Sigma = 5
file = '{0:02d}'.format(Mu) + '-' + str(Tau) + '-' + '{0:02d}'.format(Sigma)

In [5]:
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.12, right=0.95, top=0.9, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(np.arange(500), chargehist(np.arange(500)), label='$\mathrm{ChargePDF}$')
ax.set_xlabel(r'$\mathrm{Charge}/\si{mV\cdot ns}$')
ax.set_xlim(20, 400)
ax.set_ylim(0, ax.get_ylim()[1] * 1.05)
ax.set_ylabel(r'$\mathrm{PDF}$')
ax.grid()
ax.legend()
fig.savefig('Note/figures/chargehist.pgf')
fig.savefig('Note/figures/chargehist.pdf')
fig.clf()
plt.close(fig)

In [6]:
xn = np.linspace(-5, 5, 600)
yn = np.linspace(-5, 5, 600)
xv, yv = np.meshgrid(xn, yn)
pos = np.dstack((xv, yv))

m = np.array([[-1., 1.5], [-1., 1.]])
s = np.array([[1., .5], [.6, .8]])
a = np.array([[.4, .6], [.4, .6]])
rho = np.array([[0.5, -0.7], [0.4, 0.4]])

ymax = max(np.max(normcombine(xn, m[0, :], s[0, :], a[0, :])), np.max(normcombine(yn, m[1, :], s[1, :], a[1, :])))
fig = plt.figure(figsize=(5, 5))
ax0 = fig.add_axes((.1, .85, .75, .12))
ax0.axis('off')
ax0.plot(xn, normcombine(xn, m[0, :], s[0, :], a[0, :]), color='b')
ax0.fill_between(xn, normcombine(xn, m[0, :], s[0, :], a[0, :]), facecolor='blue', alpha=0.5, color=None)
ax0.set_xticks([])
ax0.set_yticks([])
ax0.set_ylim(0, ymax)
ax1 = fig.add_axes((.85, .1, .12, .75))
rot = transforms.Affine2D().rotate_deg(-90)
base = plt.gca().transData
ax1.axis('off')
ax1.plot(yn, normcombine(yn, m[1, :], s[1, :], a[1, :]), color='b', transform=rot+base)
ax1.fill_between(yn, normcombine(yn, m[1, :], s[1, :], a[1, :]), facecolor='blue', alpha=0.5, color=None, transform=rot+base)
ax1.set_xticks([])
ax1.set_yticks([])
ax1.set_xlim(0, ymax)
ax2 = fig.add_axes((.1, .1, .75, .75))
ax2.contourf(xv, yv, normcombine2d(pos, m, s, a, rho), cmap=cm.PuBu)
ax2.set_xticks([])
ax2.set_yticks([])
ax2.set_xlabel(r'$\mu$')
ax2.set_ylabel(r'$\nu$')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
fig.savefig('Note/figures/wdistdemo.pgf')
fig.savefig('Note/figures/wdistdemo.pdf')
fig.clf()
plt.close(fig)

In [7]:
t = np.array([0, 1, 2, 3])
a = np.array([0.5, 0.5, 0.5, 1])
b1 = np.array([0.5, 0.5, 1, 0.5])
b2 = np.array([1, 0.5, 0.5, 0.5])
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(2, 2, figure=fig, left=0.12, right=0.95, top=0.97, bottom=0.15, wspace=0.4, hspace=0.5)
ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[0, 1])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])
ax0.bar(t, a, color='r')
ax0.set_xlabel(r'$a$')
ax0.set_ylabel(r'$\mathrm{weight}\ a$')
ax0.set_xticks(t)
ax0.set_ylim(0, ax0.get_ylim()[1] * 1.05)
ax1.bar(t, b1, color='b')
ax1.set_xlabel(r'$b_{1}$')
ax1.set_ylabel(r'$\mathrm{weight}\ b_{1}$')
ax1.set_xticks(t)
ax1.set_ylim(0, ax1.get_ylim()[1] * 1.05)
ax2.bar(t, b2, color='b')
ax2.set_xlabel(r'$b_{2}$')
ax2.set_ylabel(r'$\mathrm{weight}\ b_{2}$')
ax2.set_xticks(t)
ax2.set_ylim(0, ax2.get_ylim()[1] * 1.05)
ax3.text(0, 0, r'$L_{2}\ =\ 0.25$')
ax3.set_xlim(0, 5)
ax3.set_ylim(-1, 1)
ax3.spines['top'].set_visible(False)
ax3.spines['right'].set_visible(False)
ax3.spines['bottom'].set_visible(False)
ax3.spines['left'].set_visible(False)
ax3.get_xaxis().set_visible(False)
ax3.get_yaxis().set_visible(False)
fig.savefig('Note/figures/tab.pgf')
fig.savefig('Note/figures/tab.pdf')
fig.clf()
plt.close(fig)

In [8]:
with h5py.File('waveform/' + file + '.h5', 'r', libver='latest', swmr=True) as ipt:
    ent = ipt['Readout/Waveform'][:]
    tru = ipt['SimTriggerInfo/PEList'][:]
    gmu = ipt['SimTriggerInfo/PEList'].attrs['gmu']
    gsigma = ipt['SimTriggerInfo/PEList'].attrs['gsigma']
    t0truth = ipt['SimTruth/T'][:]
    npe = ipt['SimTruth/T'].attrs['npe']

In [9]:
i = 5
cid = ent[i]['ChannelID']
eid = ent[i]['TriggerNo']
truth = np.sort(tru[(tru['TriggerNo'] == eid) & (tru['PMTId'] == cid)], kind='stable', order=['TriggerNo', 'PMTId', 'HitPosInWindow'])
wave = ent[i]['Waveform'].astype(np.float) * spe_pre[ent[i]['ChannelID']]['epulse']
df = pd.DataFrame(truth)
df = df.rename(columns={'HitPosInWindow':'HitTime'})
charge = df['Charge'].copy()
hittime = df['HitTime'].copy()
df = df.astype({'Charge': 'float32'})
df = df.astype({'TriggerNo' : 'str', 'PMTId' : 'str', 'HitTime' : 'str', 'Charge': 'str'})
df['HitTime'] = ['{:.02f}'.format(s) for s in hittime]
df['Charge'] = ['{:.02f}'.format(s) for s in charge]
df

,TriggerNo,PMTId,HitTime,Charge
0,5,0,295.50,166.55
1,5,0,295.74,176.55
2,5,0,296.64,183.85
3,5,0,309.02,151.88
4,5,0,320.47,171.44
5,5,0,333.09,178.87
6,5,0,343.92,172.08
7,5,0,359.12,141.93


In [10]:
ind = np.argwhere(wave > spe_pre[cid]['std'] * 5).flatten()
xmin = ((ind.min() - spe_pre[cid]['mar_l']) // 50 - 1) * 50
xmax = max(((ind.max() + spe_pre[cid]['mar_r']) // 50 + 1) * 50, xmin + 400)

In [11]:
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax2 = ax.twinx()
ax2.vlines(truth['HitPosInWindow'].min(), 0, wave.sum(), color='r', label='Record')
ax.plot(wave, label='Waveform')
ax.hlines(2, 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}/\si{mV\cdot ns}$')
rise = truth['HitPosInWindow'].min()
vsum = wave.sum()
ax2.annotate(fr'${rise:.{4}}\mathrm{{ns}}, {vsum:.{4}}\mathrm{{mV}}\cdot\mathrm{{ns}}$', xy=(truth['HitPosInWindow'].min(), wave.sum()*2/3), xytext=(truth['HitPosInWindow'].min()+120, wave.sum()*1/2), arrowprops=dict(facecolor='k', shrink=0.1, width=0.1, headwidth=2))
ax.set_ylim(-5, ax.get_ylim()[1] * 1.05)
ax2.set_ylim(-5, ax2.get_ylim()[1] * 1.05)
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/previous.pgf')
fig.savefig('Note/figures/previous.pdf')
fig.clf()
plt.close(fig)

In [12]:
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(wave, label='Waveform')
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax.set_xlim(0, len(wave))
ax.set_ylim(-5, ax.get_ylim()[1] * 1.05)
ax.legend()
fig.savefig('Note/figures/wave.pgf')
fig.savefig('Note/figures/wave.pdf')
fig.clf()
plt.close(fig)

fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.vlines(truth['HitPosInWindow'], 0, truth['Charge'], color='r', label='Charge')
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Charge}/\si{mV\cdot ns}$')
ax.set_xlim(0, len(wave))
ax.legend()
fig.savefig('Note/figures/charge.pgf')
fig.savefig('Note/figures/charge.pdf')
fig.clf()
plt.close(fig)

In [13]:
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax2 = ax.twinx()
ax2.vlines(truth['HitPosInWindow'], 0, truth['Charge'], color='r', label='Charge')
ax.plot(wave, label='Waveform')
ax.hlines(2, 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}/\si{mV\cdot ns}$')
ax.set_ylim(bottom=-5)
ax2.set_ylim(bottom=-5)
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/goal.pgf')
fig.savefig('Note/figures/goal.pdf')
fig.clf()
plt.close(fig)

In [14]:
print(wave.sum())
print(truth['Charge'][truth['Charge'] > 0].sum()) # made by noise
truth['Charge']

1386.0
1343.1537614454037


array([166.54973633, 176.54903018, 183.84769943, 151.88306248,
       171.43906828, 178.87284266, 172.07828567, 141.93403642])

In [15]:
t = np.load('result/takara/char/Channel00/testing_record_2021-03-05_02:54:28.npz')['arr_0']
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.plot(np.arange(1, len(t)+1), t, label='W-distance', color='C1')
ax.set_xlabel(r'$\mathrm{epoch}$')
ax.set_ylabel(r'$\mathrm{Wasserstein\ Distance}/\si{ns}$')
ax.legend()
ax.grid()
fig.savefig('Note/figures/epoch.pgf')
fig.savefig('Note/figures/epoch.pdf')
fig.clf()
plt.close(fig)

In [16]:
pet, pwe = wff.threshold(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['threshold'])
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
ax2.annotate('', xy=(pet.mean(), pwe.max()*1.1), xytext=(pet.mean()+pet.ptp(), pwe.max()*1.1), arrowprops=dict(facecolor='k', shrink=0.01, width=2, headwidth=4))
ax2.set_ylim(top=pwe.max()*1.2)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}/\si{mV\cdot ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/threshold.pgf')
fig.savefig('Note/figures/threshold.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)
t0 = wff.likelihoodt0(pet, char=pwe, gmu=gmu, gsigma=gsigma, Tau=Tau, Sigma=Sigma, npe=npe, s0 = spe_pre[0]['std'] / np.linalg.norm(spe_pre[0]['spe']), mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 8
truth HitPosInWindow = [295.50333407 295.7367554  296.63968918 309.01656847 320.47469584
 333.08743987 343.92012324 359.11855235], Weight = [166.54973633 176.54903018 183.84769943 151.88306248 171.43906828
 178.87284266 172.07828567 141.93403642]
truth Resi-norm = 33.769813836922516
HitPosInWindow = [292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309
 310 311 317 318 319 320 321 322 332 333 334 335 341 342 343 344 345 346
 359], Weight = [28.54462243 47.57437071 61.84668192 71.36155606 74.53318078 72.94736842
 65.01830664 58.67505721 50.74599542 44.402746   41.23112128 33.3020595
 28.54462243 31.71624714 33.3020595  34.88787185 34.88787185 33.3020595
 30.13043478 28.54462243 26.95881007 26.95881007 30.13043478 31.71624714
 28.54462243 26.95881007 28.54462243 26.95881007 26.95881007 26.95881007
 26.95881007 30.13043478 30.13043478 31.71624714 28.54462243 26.95881007
 25.37299771]
wdist = 7.433702272003364, cdiff = 42.84623855459631
Resi-norm = 77.12028544

In [17]:
pet, pwe = wff.findpeak(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['findpeak'])
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=1.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
loc = pet + spe_pre[cid]['peak_c']
loc = loc[loc < window]
amp = wave[loc]
for i in range(len(loc)):
    ax.annotate('', xy=(loc[i], amp[i]+5), xytext=(loc[i], amp[i]+15), arrowprops=dict(facecolor='k', shrink=0.01, width=0.5, headwidth=2))
ax2.annotate('', xy=(pet.mean(), pwe.max()*1.1), xytext=(pet.mean()+pet.ptp(), pwe.max()*1.1), arrowprops=dict(facecolor='k', shrink=0.01, width=2, headwidth=4))
ax2.set_ylim(top=pwe.max()*1.2)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}/\si{mV\cdot ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/findpeak.pgf')
fig.savefig('Note/figures/findpeak.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)
t0 = wff.likelihoodt0(pet, char=pwe, gmu=gmu, gsigma=gsigma, Tau=Tau, Sigma=Sigma, npe=npe, s0 = spe_pre[0]['std'] / np.linalg.norm(spe_pre[0]['spe']), mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 8
truth HitPosInWindow = [295.50333407 295.7367554  296.63968918 309.01656847 320.47469584
 333.08743987 343.92012324 359.11855235], Weight = [166.54973633 176.54903018 183.84769943 151.88306248 171.43906828
 178.87284266 172.07828567 141.93403642]
truth Resi-norm = 33.769813836922516
HitPosInWindow = [297 309 321 334 345 360], Weight = [472.26666667 215.6        184.8        174.53333333 184.8
 154.        ]
wdist = 1.7939204494244763, cdiff = 42.84623855459631
Resi-norm = 47.31981398597246
3.2908261947501956


In [18]:
pet, pwe = wff.waveformfft(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['fftrans'])
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}/\si{mV\cdot ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/fftrans.pgf')
fig.savefig('Note/figures/fftrans.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)
t0 = wff.likelihoodt0(pet, char=pwe, gmu=gmu, gsigma=gsigma, Tau=Tau, Sigma=Sigma, npe=npe, s0 = spe_pre[0]['std'] / np.linalg.norm(spe_pre[0]['spe']), mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 8
truth HitPosInWindow = [295.50333407 295.7367554  296.63968918 309.01656847 320.47469584
 333.08743987 343.92012324 359.11855235], Weight = [166.54973633 176.54903018 183.84769943 151.88306248 171.43906828
 178.87284266 172.07828567 141.93403642]
truth Resi-norm = 33.769813836922516
HitPosInWindow = [292 293 294 295 296 297 298 299 300 302 303 304 307 308 309 310 311 318
 319 320 321 322 323 330 331 332 333 334 335 342 343 344 345 346 347 356
 357 358 359 360 361], Weight = [24.82081581 48.87949505 71.83935731 88.68732369 95.39227523 90.03762625
 73.43659145 49.03142389 22.08071804 17.3662736  22.39675782 16.84392005
 28.56039852 37.98377877 39.41949121 33.02714783 21.15264523 22.08172329
 31.7856764  36.48129781 35.11598446 28.27541313 17.92967789 15.82610382
 23.92024064 28.88120272 29.70816756 26.46764181 20.22486538 18.56370459
 25.74806945 30.53548604 31.58364313 28.41504487 21.5404445  16.79769708
 23.04605969 26.24442826 26.00128452 22.67260556 17.19749767]
wdist = 2.

In [19]:
pet, pwe = wff.lucyddm(wave, spe_pre[cid]['spe'])
pet, pwe = wff.clip(pet, pwe, Thres['lucyddm'])
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}/\si{mV\cdot ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/lucyddm.pgf')
fig.savefig('Note/figures/lucyddm.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)
t0 = wff.likelihoodt0(pet, char=pwe, gmu=gmu, gsigma=gsigma, Tau=Tau, Sigma=Sigma, npe=npe, s0 = spe_pre[0]['std'] / np.linalg.norm(spe_pre[0]['spe']), mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 8
truth HitPosInWindow = [295.50333407 295.7367554  296.63968918 309.01656847 320.47469584
 333.08743987 343.92012324 359.11855235], Weight = [166.54973633 176.54903018 183.84769943 151.88306248 171.43906828
 178.87284266 172.07828567 141.93403642]
truth Resi-norm = 33.769813836922516
HitPosInWindow = [295 296 309 320 321 332 333 344 359], Weight = [ 51.24098716 521.84352709 154.55047212 103.40783981  91.86134644
  38.02591939  89.61884416 178.68563188 156.76543196]
wdist = 1.395328707539214, cdiff = 42.84623855459631
Resi-norm = 37.16513364429412
1.1501016482755517


In [20]:
pet, pwe = wff.xiaopeip(wave, spe_pre[cid])
pet, pwe = wff.clip(pet, pwe, Thres['xiaopeip'])
pwe = pwe / pwe.sum() * np.abs(wave.sum())
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}/\si{mV\cdot ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/xiaopeip.pgf')
fig.savefig('Note/figures/xiaopeip.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)
t0 = wff.likelihoodt0(pet, char=pwe, gmu=gmu, gsigma=gsigma, Tau=Tau, Sigma=Sigma, npe=npe, s0 = spe_pre[0]['std'] / np.linalg.norm(spe_pre[0]['spe']), mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 8
truth HitPosInWindow = [295.50333407 295.7367554  296.63968918 309.01656847 320.47469584
 333.08743987 343.92012324 359.11855235], Weight = [166.54973633 176.54903018 183.84769943 151.88306248 171.43906828
 178.87284266 172.07828567 141.93403642]
truth Resi-norm = 33.769813836922516
HitPosInWindow = [295 296 297 309 320 321 333 344 359], Weight = [100.67115849 421.36471577  55.39451108 161.11597679 104.22846356
  87.19521003 111.31215768 190.04447858 154.67332802]
wdist = 1.577299292694693, cdiff = 42.84623855459631
Resi-norm = 38.21810075479327
1.1116689572257314


In [21]:
p = spe_pre[cid]['parameters']
nsp = 4
nstd = 3
D = 1000
factor = np.linalg.norm(spe_pre[cid]['spe'])
n = max(math.ceil(Mu / math.sqrt(Tau ** 2 + Sigma ** 2)) * 10, 1)
A, wave_r, tlist, t0_init, t0_init_delta, char_init = wff.initial_params(wave, spe_pre[cid], Mu, Tau, Sigma, gmu, gsigma, 0.2, npe, p, nsp, nstd, is_t0=True, is_delta=False, n=n, nshannon=1)
A = A / factor
xmmse, xmmse_star, psy_star, nu_star, T_star, d_tot_i, d_max_i = wff.fbmpr_fxn_reduced(wave_r, A, min(-1e-3+1, Mu / len(tlist)), spe_pre[cid]['std'] ** 2, (gsigma * factor / gmu) ** 2, factor, D, stop=0)
xmmse_most = xmmse_star[0]
pet = tlist[xmmse_most > 0]
cha = xmmse_most[xmmse_most > 0] / factor
pet, pwe = wff.clip(pet, cha, 0.0)
pwe = pwe * gmu
fig = plt.figure()
# fig.tight_layout()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.vlines(pet, 0, pwe, color='r', label='Charge', linewidth=0.5)
ax.plot(wave, label='Waveform')
ax.hlines(5 * spe_pre[cid]['std'], 0, window, color='g', label='Threshold')
ax.set_xlim(xmin, xmax)
lines, labels = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2)
ax.set_xlabel(r'$\mathrm{t}/\si{ns}$')
ax.set_ylabel(r'$\mathrm{Voltage}/\si{mV}$')
ax2.set_ylabel(r'$\mathrm{Charge}/\si{mV\cdot ns}$')
align.yaxes(ax, 0, ax2, 0)
fig.savefig('Note/figures/fbmp.pgf')
fig.savefig('Note/figures/fbmp.pdf')
fig.clf()
plt.close(fig)
wff.demo(pet, pwe, truth, spe_pre[cid], window, wave, cid, p)
t0 = wff.likelihoodt0(pet, char=pwe, gmu=gmu, gsigma=gsigma, Tau=Tau, Sigma=Sigma, npe=npe, s0 = spe_pre[0]['std'] / np.linalg.norm(spe_pre[0]['spe']), mode='charge')[0]
print((t0 - t0truth[i]['T0']).item())

PEnum is 8
truth HitPosInWindow = [295.50333407 295.7367554  296.63968918 309.01656847 320.47469584
 333.08743987 343.92012324 359.11855235], Weight = [166.54973633 176.54903018 183.84769943 151.88306248 171.43906828
 178.87284266 172.07828567 141.93403642]
truth Resi-norm = 33.769813836922516
HitPosInWindow = [295.5 296.  296.3 309.2 320.3 333.2 344.  358.9], Weight = [177.84021776 173.97857565 174.87974059 155.47775465 176.16241508
 169.86353881 179.98205947 143.96640197]
wdist = 0.31483208021832854, cdiff = 8.996942533520496
Resi-norm = 33.59171708887417
1.4941382217205046


In [22]:
with h5py.File('result/threshold/dist/' + file + '.h5', 'r', libver='latest', swmr=True) as distfile:
    dt = distfile['Record'][:]
fig = plt.figure()
# fig.tight_layout()
gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
ax = fig.add_subplot(gs[0, 0])
ax.hist(dt['NPE'], bins=np.max(dt['NPE']), histtype='step')
ax.set_yscale('log')
ax.grid()
ax.set_xlabel(r'$N_{pe}$')
ax.set_ylabel(r'$\mathrm{Count}$')
fig.savefig('Note/figures/penum.pgf')
fig.savefig('Note/figures/penum.pdf')
plt.close(fig)

In [23]:
with h5py.File('result/mcmc/solu/' + file + '.h5', 'r', libver='latest', swmr=True) as soluf, h5py.File('waveform/' + file + '.h5', 'r', libver='latest', swmr=True) as wavef:
    start = wavef['SimTruth/T'][:]
    time = soluf['starttime'][:]
data = time['tswave'] - start['T0']
fig = plt.figure()
ax = fig.add_axes((.1, .45, .85, .45))
ax.hist(data, bins=100, density=1)
ax.set_ylabel('Normalized Count')
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlim(data.min() - 0.05, data.max() + 0.05)
ax.set_ylim(0, ax.get_ylim()[1] * 1.05)
axb = fig.add_axes((.1, .15, .85, .3))
axb.boxplot(data, vert=False, sym='', patch_artist=True)
axb.set_xlabel(r'$\Delta t_{0}/\si{ns}$')
axb.set_yticks([])
axb.set_xlim(ax.get_xlim())
fig.savefig('Note/figures/mcmcrect0hist.pgf')
fig.savefig('Note/figures/mcmcrect0hist.pdf')
plt.close(fig)

In [40]:
methods = ['lucyddm', 'xiaopeip', 'takara', 'fbmp', 'mcmc']

for m in methods:
    with h5py.File('result/' + m + '/dist/' + file + '.h5', 'r', libver='latest', swmr=True) as distfile:
        dt = distfile['Record'][:]
    N = np.percentile(dt['wdist'], 95)
    M = 500

    penum = np.unique(dt['NPE'])
    l = min(50, penum.max())
    wdist_stats = np.full((l, 6), np.nan)
    edist_stats = np.full((l, 6), np.nan)
    for i in range(l):
        vali = dt['NPE'] == i+1
        if np.sum(vali) == 0:
            continue
        dtwpi = dt['wdist'][vali]
        dtepi = dt['RSS'][vali]
        wdist_stats[i, 0] = np.median(dtwpi)
        wdist_stats[i, 1] = np.median(np.abs(dtwpi - np.median(dtwpi)))
        wdist_stats[i, 2] = np.mean(dtwpi)
        wdist_stats[i, 3] = np.std(dtwpi)
        wdist_stats[i, 4] = np.percentile(dtwpi, 5)
        wdist_stats[i, 5] = np.percentile(dtwpi, 95)
        edist_stats[i, 0] = np.median(dtepi)
        edist_stats[i, 1] = np.median(np.abs(dtepi - np.median(dtepi)))
        edist_stats[i, 2] = np.mean(dtepi)
        edist_stats[i, 3] = np.std(dtepi)
        edist_stats[i, 4] = np.percentile(dtepi, 5)
        edist_stats[i, 5] = np.percentile(dtepi, 95)

#     n = 4 # corresponding with Algorithm.tex
    L = len(dt)
    fig = plt.figure()
    ax = fig.add_axes((.1, .45, .85, .45))
    data = dt['wdist']
    vali = np.abs(data - np.mean(data)) < 7 * np.std(data, ddof=-1)
    data = data[vali]
    ax.hist(data, bins=np.arange(0, data.max(), np.percentile(data, 95) / 50), density=1)
    ax.set_ylabel('Normalized Count')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim(data.min() - 0.05, np.percentile(data, 98) + 0.05)
    ax.set_ylim(0, ax.get_ylim()[1] * 1.05)
    axb = fig.add_axes((.1, .15, .85, .3))
    axb.boxplot(data, vert=False, sym='', patch_artist=True)
    axb.set_xlabel(r'$\mathrm{t}/\si{ns}$')
    axb.set_yticks([])
    axb.set_xlim(ax.get_xlim())
    fig.savefig('Note/figures/' + m + 'chargehist.pgf')
    fig.savefig('Note/figures/' + m + 'chargehist.pdf')
    plt.close(fig)

    fig = plt.figure()
    gs = gridspec.GridSpec(1, 1, figure=fig, left=0.15, right=0.85, top=0.95, bottom=0.15, wspace=0.4, hspace=0.5)
    ax = fig.add_subplot(gs[0, 0])
#     ey = np.vstack([wdist_stats[:, 0]-wdist_stats[:, 4], wdist_stats[:, 5]-wdist_stats[:, 0]])
#     ax.errorbar(np.arange(1, l+1), wdist_stats[:, 0], yerr=ey, label=r'$W-dist^{95\%}_{5\%}$')
    ax.boxplot(np.array([dt['wdist'][dt['NPE'] == i+1] for i in range(l)], dtype=np.object), sym='', patch_artist=True)
    ax.plot(np.arange(1, l + 1), wdist_stats[:, 0], label='W-dist')
    ax.set_xlim(-1, l + 1)
    ax.set_xlabel(r'$N_{pe}$')
    ax.set_ylabel(r'$\mathrm{W-dist}/\si{ns}$')
    # ax.set_title('W-dist vs NPE stats')
    ax.legend()
    fig.savefig('Note/figures/' + m + 'chargestats.pgf')
    fig.savefig('Note/figures/' + m + 'chargestats.pdf')
    plt.close(fig)